

   "![FMD_Overview](https://github.com/edkreuk/FMD_FRAMEWORK/blob/main/Images/FMD_Overview.png?raw=true)"

Create Lakehouse if not exists and Attach Lakehouse 

In [ ]:
try:
    notebookutils.lakehouse.create(name = "LH_FMD_CONFIGURATION")
except Exception as ex:
    print('Lakehouse already exists')

In [ ]:
%%configure -f
{ 
    "defaultLakehouse": { 
        "name":  "LH_FMD_CONFIGURATION"
    }
}

# Parameters

In [ ]:
deploy_from_github = True
driver = '{ODBC Driver 18 for SQL Server}'
# target connections guid 
# add the correct id for every connections
# change the capacity id
# change the workspace roles

FrameworkName= 'DEMO' # max 6 characters for better visibility, no spaces and the end of the name

capacity_id_dvlm = '098FA313-D63F-4F04-897E-3F25DCBC838C'       # Which capacity will be used for these workspaces in development
capacity_id_prod = '098FA313-D63F-4F04-897E-3F25DCBC838C'       # Which capacity will be used for these workspaces in production
capacity_id_config = 'CF23FF9B-FAD4-4C6A-B6C6-CFCC362CF93D'     #'Which capacity will be used for this workspace for the FMD Database


workspace_roles = [ # Keep emtpy [] if you only want to assign this to your personal account
                    {
                        "principal": {
                            "id": "5c906b5c-d1d7-4984-b047-adacd8d795fe",
                            "displayName": "sg-fmd-fabric-contributor",
                            "type": "Group"
                        },
                        "role": "Contributor"  # Admin, Member, Contributor, Viewer
                        }
                    ]


configuration = {
                    'workspace': {
                        'name' : FrameworkName + ' CONFIG FMD', # Name of target workspace
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id_config
                    },
                       'DatabaseName' : 'SQL_'+FrameworkName+'_FRAMEWORK' # Name of target configuration SQL Database
}
environments = [
                    {
                        'environment_name' : 'development', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (D) FMD', # Name of target code workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_dvlm
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (D) FMD', # Name of target data workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_dvlm
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e', # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7', # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ADF_PIPELINES' : '02e107b8-e97e-4b00-a28c-668cf9ce3d9a' # Optional Guid to an Azure Datafactory connection
                        }
                    },
                    {
                        'environment_name' : 'production', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (P) FMD',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_prod
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (P) FMD',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_prod
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',  # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',  # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ADF_PIPELINES' : '02e107b8-e97e-4b00-a28c-668cf9ce3d9a'
                        }
                    }
                ]



## Import Libraries

In [ ]:
from json import loads, dumps
import json
import requests
import base64
import time
import uuid
import struct
import pyodbc

from typing import Callable, List, Dict, Optional, Any
from datetime import datetime
from time import sleep, time
from dataclasses import dataclass, field

## Download Configuration File from Github or upload the file manually 

**Manually**: upload to Files/deployment/FMD_deployment.json

In [ ]:
# Open deployment json file
deployment_manifest = {}
if deploy_from_github:
    print("Downloading from Github to FMD_FRAMEWORK")
    url = 'https://raw.githubusercontent.com/edkreuk/FMD_FRAMEWORK/main/FMD_deployment.json'
    github_download = requests.get(url)
    folder_path = notebookutils.fs.getMountPath('/default') + "/Files/deployment/"
    notebookutils.fs.mkdirs(f"file://" +folder_path)
    with open(folder_path + "FMD_deployment.json", "w") as f:
        f.write(json.dumps(github_download.json()))
    

    print("Read from FMD_FRAMEWORK Github")


# Read deployment manifest

with open(f"{notebookutils.fs.getMountPath('/default')}/Files/deployment/FMD_deployment.json") as f:
    deployment_manifest = json.load(f)
    print("Read from FMD_FRAMEWORK LOCAL")


## Load deployment functions

In [ ]:
%run NB_FMD_DEPLOYMENT_UTILS

In [ ]:
tasks=[]

Create token and session to run notebook

In [ ]:
fmd_api_access_token =  notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api')
fabric_session = create_fabric_session(fabric_token = fmd_api_access_token)

In [ ]:
# check if token is valid
for token in [fmd_api_access_token]:
    if not token:
        continue
    header, payload, signature = token.split('.')
    payload += '=' * (-len(payload) % 4)  # Add padding
    token_dict = loads(base64.urlsafe_b64decode(payload))
    directory_id = token_dict.get("tid")
    timest = token_dict.get("exp")
    expiry = (datetime.fromtimestamp(timest) - datetime.now()).total_seconds() // 60
    expiry_str = str(expiry) if expiry < 5 else str(expiry)
    print(F"token {directory_id} will expire in {expiry_str} minutes at\t{datetime.fromtimestamp(timest)} UTC")
print(F"Current time:\t\t\t\t\t\t\t\t\t{datetime.now().replace(microsecond=0)} UTC")

## Start Deployment

In [ ]:
# Create necessary workspaces 
start = time()

for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    # Loop through the workspace names and get their IDs
    
    for workspace in [environment['workspaces']['data'], environment['workspaces']['code'], configuration['workspace']]:
        
        print(f" -----")
        print(f" - Processing: data workspace {environment['environment_name']}")
        
        # List all workspaces
        workspaces_current = get_fabric_workspaces(fabric_session)  
   
        
        # Check if the displayName exists in the workspaces
        matching_workspaces = [workspace_current for workspace_current in workspaces_current.get('value') if workspace_current['displayName'] == workspace['name']]
        
        if matching_workspaces:
            print(f" - Workspace '{workspace['name']}' found. Workspace ID: {matching_workspaces[0]['id']}")
            workspace['id'] = matching_workspaces[0]['id']
        else:
            print(f" - Workspace '{workspace['name']}' not found. Creating new workspace...")
            workspace_created = fabric_request(fabric_session, F"workspaces/", 'POST', payload={"displayName": workspace['name']}, payloadtype='json')
            workspace['id'] = workspace_created['id']
            tasks.append({"task_name":f"create item {workspace['name']} initially", "task_duration": int(time() - start), "status": "success"})
        
        assign_fabric_workspace_capacity(fabric_session, workspace['id'], workspace['capacity_id'])
        tasks.append({"task_name":f"Workspace '{workspace['name']}' connected to capacity ID: {workspace['capacity_id']}", "task_duration": int(time() - start), "status": "success"})

        assign_fabric_workspace_identity(fabric_session, workspace['id'])



        # Check if roles exists or create them
        print(f" - Assiging Workspace roles")
        assign_fabric_workspace_roles(fabric_session, workspace['id'], workspace['roles'])

    # Print the workspace IDs
    print(f"--------------------------")
    print(f"Workspace ID for data workspace {environment['environment_name']}: {environment['workspaces']['data']['id']}")
    print(f"Workspace ID for code workspace {environment['environment_name']}: {environment['workspaces']['code']['id']}")
    print(f"Workspace ID for config workspace: {configuration['workspace']['id']}")


In [ ]:
# re-map databases

for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] in ('SQLDatabase','SQLEndpoint'):
        target_item['displayName'] = configuration['DatabaseName']


In [ ]:
def create_fabric_database(configuration, deployment_manifest, environment):
    try:
        workspace_deployment(configuration['workspace'], deployment_manifest['configuration']['items'], environment['guids_to_replace'], True)
        environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_config"], "new_id": configuration['workspace']['id']})
    except Exception as e:
        print(f"Error: {e}. Retrying in 60 seconds...")
        sleep(60)
        create_fabric_database(configuration, deployment_manifest, environment)



In [ ]:
# create empty items if not exists
    
for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    environment['guids_to_replace'] = []
    workspace_deployment(environment['workspaces']['code'], deployment_manifest['items'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_code"], "new_id": environment['workspaces']['code']['id']})
    workspace_deployment(environment['workspaces']['data'], deployment_manifest['data'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_data"], "new_id": environment['workspaces']['data']['id']})


    

    

In [ ]:
#Create Fabric SQL Database
create_fabric_database(configuration, deployment_manifest, environment)

In [ ]:
items_to_deploy = deployment_manifest["items"]

for environment in environments:
    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    # Deploy items to workspace
    # Append the remaining pairs
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_SQL"], "new_id": environment['connections']['CON_FMD_FABRIC_SQL']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_PIPELINES"], "new_id": environment['connections']['CON_FMD_FABRIC_PIPELINES']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADF_PIPELINES"], "new_id": environment['connections']['CON_FMD_ADF_PIPELINES']})
    environment['guids_to_replace'].append({"old_id": "SQL_FMD_FRAMEWORK", "new_id":  configuration['DatabaseName']})
    
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    deploy_items(items_to_deploy, environment['guids_to_replace'], fmd_api_access_token, environment['workspaces']['code']['id'], existing_items["value"])

In [ ]:
#print(environment['guids_to_replace'])

In [ ]:
#Get all the connections and add them to Manifest to automatically insert them in Framework
#Make sure all connections are starting with CON_FMD to filter out
connections = get_fabric_connections(fabric_session)

for connection in connections['value']:
    display_name = connection.get('displayName', '')
    if display_name and display_name.startswith('CON_FMD'):
        connection_type = connection.get('connectionDetails', {}).get('type', 'Unknown')
        connection_id = connection.get('id')

        exec_statement = (
            f"EXEC [integration].[sp_UpsertConnection] "
            f"@ConnectionGuid = \"{connection_id}\", "
            f"@Name = \"{display_name}\", "
            f"@Type = \"{connection_type}\", "
            f"@IsActive = 1"
        )

        deployment_manifest["configuration"]["queries_stored_procedures"].append(exec_statement)



In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "00000000-0000-0000-0000-000000000000", @Name = "CON_FMD_ONELAKE", @Type = "ONELAKE", @IsActive = 1')


In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type='ONELAKE_TABLES_01')
    DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
    EXECUTE [integration].[sp_UpsertDataSource] 
        @ConnectionId = @ConnectionIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@Name = 'LH_DATA_LANDINGZONE'
        ,@Namespace = 'ONELAKE'
        ,@Type = 'ONELAKE_TABLES_01'
        ,@Description = 'ONELAKE_TABLES'
        ,@IsActive = 1
""")

In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type ='ONELAKE_FILES_01')
    DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
    EXECUTE [integration].[sp_UpsertDataSource] 
        @ConnectionId = @ConnectionIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@Name = 'LH_DATA_LANDINGZONE'
        ,@Namespace = 'ONELAKE'
        ,@Type = 'ONELAKE_FILES_01'
        ,@Description = 'ONELAKE_FILES'
        ,@IsActive = 1
""")

In [ ]:
workspaces = []
workspaces.append(configuration['workspace'])

for environment in environments:
    workspaces.append(environment['workspaces']['code'])
    workspaces.append(environment['workspaces']['data'])
    
for workspace in workspaces:
    print(f'EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "{workspace["id"]}" ,@Name = "{workspace["name"]}")')
    deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertWorkspace] @WorkspaceId = "{workspace["id"]}", @Name = "{workspace["name"]}"')

In [ ]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'DataPipeline':
            print(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

In [ ]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['data']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'Lakehouse':
            print(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type='ONELAKE_TABLES_01')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_DATA_LANDINGZONE')
    EXECUTE [integration].[sp_UpsertLandingzoneEntity] 
        @LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@SourceSchema = 'in'
        ,@SourceName = 'customer'
        ,@SourceCustomSelect = ''
        ,@FileName = 'customer'
        ,@FilePath = 'fmd'
        ,@FileType = 'parquet'
        ,@IsIncremental = 0
        ,@IsIncrementalColumn = ''
        ,@IsActive = 1
""")

In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_BRONZE_LAYER')
    EXECUTE [integration].[sp_UpsertBronzeLayerEntity] 
        @BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@Schema = 'in'
        ,@Name = 'customer'
        ,@FileType = 'Delta'
        ,@LakehouseId = @LakehouseIdInternal
        ,@PrimaryKeys = 'CustomerId'
        ,@IsActive = 1
""")

In [ ]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @SilverLayerEntityIdInternal INT = (SELECT SilverLayerEntityId FROM integration.SilverLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_SILVER_LAYER')
    EXECUTE [integration].[sp_UpsertSilverLayerEntity] 
        @SilverLayerEntityId = @SilverLayerEntityIdInternal
        ,@BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@Name = 'customer'
        ,@Schema = 'in'
        ,@FileType = 'delta'
        ,@IsActive = 1
""")

In [ ]:
for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] == 'SQLDatabase':
        connstring = target_item["connectionString"]
        database = target_item['databaseName']

try:
    i = 0
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 10  # Setting a lower timeout for subsequent queries
        for i, query in enumerate(deployment_manifest["configuration"]["queries_tables"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(deployment_manifest["configuration"]["queries_views"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(deployment_manifest["configuration"]["queries_stored_procedures"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(deployment_manifest["configuration"]["queries_logging"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"success"})
except pyodbc.OperationalError as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

In [ ]:
display(tasks)